#https://www.kaggle.com/c/invasive-species-monitoring/data  

pip install kaggle-cli  
#get data from kaggle   
kg download -u -p -c invasive-species-monitoring  
#to see contents of file  
7z l train.7z  
#to unzip  
7z x train.7z  
7z x test.7z  
unzip sample_submission.csv.zip   
unzip train_labels.csv.zip  
#to view results  
tree -d  


In [1]:
from datetime import datetime
HOMEPATH = "/home/ubuntu/fastai/"
DATA_HOME_DIR = HOMEPATH + "data/invasive/"

validFract = 0.3
sampleFract = 0.2
import os

In [2]:
os.chdir(HOMEPATH)
print ("os.getcwd:", os.getcwd())
# Rather than importing everything manually, we'll make things easy
#   and load them all in utils.py, and just import them from there.
import utils;
from utils import *
from shutil import copyfile

import numpy as np
import pandas as pd
from random import shuffle
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

('os.getcwd:', '/home/ubuntu/fastai')


WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5110)
Using Theano backend.


In [3]:
TRAIN  = DATA_HOME_DIR + "train/"
VALID  = DATA_HOME_DIR + "valid/"
TEST   = DATA_HOME_DIR + "test/"
RESULTS = DATA_HOME_DIR + "results/"
SAMPLE =  DATA_HOME_DIR + "sample/"
SAMPLE_TRAIN = SAMPLE + "train/"
SAMPLE_VALID = SAMPLE + "valid/"
SAMPLE_TEST = SAMPLE + "test/"
SAMPLE_RESULTS = SAMPLE + "results/"


In [4]:
def dirFileList(dir_path):
    return [name for name in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, name))]


In [5]:
dirList = [TRAIN, TEST]

for dir_path in dirList:
    print ("# of files in", dir_path, ":", len(dirFileList(dir_path)))
os.chdir(DATA_HOME_DIR)
print ("os.getcwd:", os.getcwd())
!tree -d

('# of files in', '/home/ubuntu/fastai/data/invasive/train/', ':', 0)
('# of files in', '/home/ubuntu/fastai/data/invasive/test/', ':', 0)
('os.getcwd:', '/home/ubuntu/fastai/data/invasive')
.
├── results
├── sample
│   ├── results
│   ├── test
│   │   └── unknown
│   ├── train
│   │   ├── harmless
│   │   └── invasive
│   └── valid
│       ├── harmless
│       └── invasive
├── test
│   └── unknown
├── train
│   ├── harmless
│   └── invasive
└── valid
    ├── harmless
    └── invasive

19 directories


In [10]:
#check if dir exists, create if not already.
def makeNewDir(newDirPath):
    if not os.path.exists(newDirPath): 
        print("directory ", newDirPath, " did not exist, creating:")
        os.mkdir(newDirPath)
    else:
        print("directory ", newDirPath, "already existed. filecount = ", len(dirFileList(newDirPath)))


In [31]:
os.chdir(DATA_HOME_DIR)
print ("os.getcwd:", os.getcwd())
categories = ['harmless/', 'invasive/']#nb: do not change order or trailing slash
UNKNOWN = 'unknown/'

#NB: train and test are created by unzipping the data.
makeNewDir(SAMPLE)
makeNewDir(SAMPLE_TRAIN)
makeNewDir(SAMPLE_VALID)
makeNewDir(VALID)
makeNewDir(TEST+UNKNOWN)
makeNewDir(SAMPLE_TEST)
makeNewDir(SAMPLE_TEST+UNKNOWN)
makeNewDir(RESULTS)
makeNewDir(SAMPLE_RESULTS)


for category in categories:
    makeNewDir(TRAIN+category)
    makeNewDir(VALID+category)
    makeNewDir(SAMPLE_TRAIN+category)
    makeNewDir(SAMPLE_VALID+category)
!tree -d

('os.getcwd:', '/home/ubuntu/fastai/data/invasive')
('directory ', '/home/ubuntu/fastai/data/invasive/sample/', 'already existed. filecount = ', 0)
('directory ', '/home/ubuntu/fastai/data/invasive/sample/train/', 'already existed. filecount = ', 0)
('directory ', '/home/ubuntu/fastai/data/invasive/sample/valid/', 'already existed. filecount = ', 0)
('directory ', '/home/ubuntu/fastai/data/invasive/valid/', 'already existed. filecount = ', 0)
('directory ', '/home/ubuntu/fastai/data/invasive/test/unknown/', 'already existed. filecount = ', 1531)
('directory ', '/home/ubuntu/fastai/data/invasive/sample/test/', 'already existed. filecount = ', 0)
('directory ', '/home/ubuntu/fastai/data/invasive/sample/test/unknown/', 'already existed. filecount = ', 306)
('directory ', '/home/ubuntu/fastai/data/invasive/results/', ' did not exist, creating:')
('directory ', '/home/ubuntu/fastai/data/invasive/sample/results/', ' did not exist, creating:')
('directory ', '/home/ubuntu/fastai/data/invasive

In [12]:
#load trainLabels.csv and use to move files to category directories
trainLabelsFile = DATA_HOME_DIR + "train_labels.csv"
trainLabels = pd.read_csv(trainLabelsFile).as_matrix()
print (type(trainLabels), trainLabels.shape)
print (trainLabels[0:10,])

(<type 'numpy.ndarray'>, (2295, 2))
[[ 1  0]
 [ 2  0]
 [ 3  1]
 [ 4  0]
 [ 5  1]
 [ 6  0]
 [ 7  1]
 [ 8  1]
 [ 9  0]
 [10  0]]


In [13]:
#first move: from train to train/categories using categories nominated in train_labels.csv
success = 0
error = 0
count = 0
for i in range(0, trainLabels.shape[0]):
    print (trainLabels[i,], trainLabels[i,1])
    filename = TRAIN+str(trainLabels[i,0])+".jpg"
    #print("filename:", filename)
    #img=mpimg.imread(filename)
    #imgplot = plt.imshow(img)
    #break
    if trainLabels[i,1]==0:
        #move to harmless
        dest = TRAIN+categories[0]+str(trainLabels[i,0])+".jpg"
    else:
        #move to invasive
        dest = TRAIN+categories[1]+str(trainLabels[i,0])+".jpg"
    try:
        print ("moving from ", filename, " to ", dest)
        os.rename(filename, dest)
        success += 1
    except:
        print("error moving :", filename)
        error += 1
    count += 1
    #if count==5: break
print ("success:", success)
print ("error:", error)
print ("count:", count)

(array([1, 0]), 0)
('moving from ', '/home/ubuntu/fastai/data/invasive/train/1.jpg', ' to ', '/home/ubuntu/fastai/data/invasive/train/harmless/1.jpg')
(array([2, 0]), 0)
('moving from ', '/home/ubuntu/fastai/data/invasive/train/2.jpg', ' to ', '/home/ubuntu/fastai/data/invasive/train/harmless/2.jpg')
(array([3, 1]), 1)
('moving from ', '/home/ubuntu/fastai/data/invasive/train/3.jpg', ' to ', '/home/ubuntu/fastai/data/invasive/train/invasive/3.jpg')
(array([4, 0]), 0)
('moving from ', '/home/ubuntu/fastai/data/invasive/train/4.jpg', ' to ', '/home/ubuntu/fastai/data/invasive/train/harmless/4.jpg')
(array([5, 1]), 1)
('moving from ', '/home/ubuntu/fastai/data/invasive/train/5.jpg', ' to ', '/home/ubuntu/fastai/data/invasive/train/invasive/5.jpg')
(array([6, 0]), 0)
('moving from ', '/home/ubuntu/fastai/data/invasive/train/6.jpg', ' to ', '/home/ubuntu/fastai/data/invasive/train/harmless/6.jpg')
(array([7, 1]), 1)
('moving from ', '/home/ubuntu/fastai/data/invasive/train/7.jpg', ' to ', '

(array([609,   1]), 1)
('moving from ', '/home/ubuntu/fastai/data/invasive/train/609.jpg', ' to ', '/home/ubuntu/fastai/data/invasive/train/invasive/609.jpg')
(array([610,   1]), 1)
('moving from ', '/home/ubuntu/fastai/data/invasive/train/610.jpg', ' to ', '/home/ubuntu/fastai/data/invasive/train/invasive/610.jpg')
(array([611,   1]), 1)
('moving from ', '/home/ubuntu/fastai/data/invasive/train/611.jpg', ' to ', '/home/ubuntu/fastai/data/invasive/train/invasive/611.jpg')
(array([612,   1]), 1)
('moving from ', '/home/ubuntu/fastai/data/invasive/train/612.jpg', ' to ', '/home/ubuntu/fastai/data/invasive/train/invasive/612.jpg')
(array([613,   1]), 1)
('moving from ', '/home/ubuntu/fastai/data/invasive/train/613.jpg', ' to ', '/home/ubuntu/fastai/data/invasive/train/invasive/613.jpg')
(array([614,   1]), 1)
('moving from ', '/home/ubuntu/fastai/data/invasive/train/614.jpg', ' to ', '/home/ubuntu/fastai/data/invasive/train/invasive/614.jpg')
(array([615,   1]), 1)
('moving from ', '/home

('moving from ', '/home/ubuntu/fastai/data/invasive/train/1239.jpg', ' to ', '/home/ubuntu/fastai/data/invasive/train/harmless/1239.jpg')
(array([1240,    0]), 0)
('moving from ', '/home/ubuntu/fastai/data/invasive/train/1240.jpg', ' to ', '/home/ubuntu/fastai/data/invasive/train/harmless/1240.jpg')
(array([1241,    1]), 1)
('moving from ', '/home/ubuntu/fastai/data/invasive/train/1241.jpg', ' to ', '/home/ubuntu/fastai/data/invasive/train/invasive/1241.jpg')
(array([1242,    1]), 1)
('moving from ', '/home/ubuntu/fastai/data/invasive/train/1242.jpg', ' to ', '/home/ubuntu/fastai/data/invasive/train/invasive/1242.jpg')
(array([1243,    1]), 1)
('moving from ', '/home/ubuntu/fastai/data/invasive/train/1243.jpg', ' to ', '/home/ubuntu/fastai/data/invasive/train/invasive/1243.jpg')
(array([1244,    1]), 1)
('moving from ', '/home/ubuntu/fastai/data/invasive/train/1244.jpg', ' to ', '/home/ubuntu/fastai/data/invasive/train/invasive/1244.jpg')
(array([1245,    1]), 1)
('moving from ', '/hom

(array([1869,    1]), 1)
('moving from ', '/home/ubuntu/fastai/data/invasive/train/1869.jpg', ' to ', '/home/ubuntu/fastai/data/invasive/train/invasive/1869.jpg')
(array([1870,    1]), 1)
('moving from ', '/home/ubuntu/fastai/data/invasive/train/1870.jpg', ' to ', '/home/ubuntu/fastai/data/invasive/train/invasive/1870.jpg')
(array([1871,    0]), 0)
('moving from ', '/home/ubuntu/fastai/data/invasive/train/1871.jpg', ' to ', '/home/ubuntu/fastai/data/invasive/train/harmless/1871.jpg')
(array([1872,    1]), 1)
('moving from ', '/home/ubuntu/fastai/data/invasive/train/1872.jpg', ' to ', '/home/ubuntu/fastai/data/invasive/train/invasive/1872.jpg')
(array([1873,    1]), 1)
('moving from ', '/home/ubuntu/fastai/data/invasive/train/1873.jpg', ' to ', '/home/ubuntu/fastai/data/invasive/train/invasive/1873.jpg')
(array([1874,    1]), 1)
('moving from ', '/home/ubuntu/fastai/data/invasive/train/1874.jpg', ' to ', '/home/ubuntu/fastai/data/invasive/train/invasive/1874.jpg')
(array([1875,    0]), 

In [14]:
def moveFiles(fileList, orig, dest):
    success = 0
    error = 0
    count = 0
    for f in fileList:
        filename = orig+f
        destFilename = dest+f
        try:
            print "moving from ", filename+" to ", destFilename
            os.rename(filename, destFilename)
            success += 1
        except:
            print("error moving :", filename)
            error += 1
        count += 1
        #break
    print ("success:", success)
    print ("error:", error)
    print ("count:", count)


In [15]:
#move files from test to test/unknown
os.chdir(TEST)
print ("os.getcwd:", os.getcwd())
files = glob('*.jpg')
print len(files)

moveFiles(files, TEST, TEST+UNKNOWN)



('os.getcwd:', '/home/ubuntu/fastai/data/invasive/test')
1531
moving from  /home/ubuntu/fastai/data/invasive/test/1516.jpg to  /home/ubuntu/fastai/data/invasive/test/unknown/1516.jpg
moving from  /home/ubuntu/fastai/data/invasive/test/314.jpg to  /home/ubuntu/fastai/data/invasive/test/unknown/314.jpg
moving from  /home/ubuntu/fastai/data/invasive/test/341.jpg to  /home/ubuntu/fastai/data/invasive/test/unknown/341.jpg
moving from  /home/ubuntu/fastai/data/invasive/test/1395.jpg to  /home/ubuntu/fastai/data/invasive/test/unknown/1395.jpg
moving from  /home/ubuntu/fastai/data/invasive/test/1483.jpg to  /home/ubuntu/fastai/data/invasive/test/unknown/1483.jpg
moving from  /home/ubuntu/fastai/data/invasive/test/943.jpg to  /home/ubuntu/fastai/data/invasive/test/unknown/943.jpg
moving from  /home/ubuntu/fastai/data/invasive/test/1457.jpg to  /home/ubuntu/fastai/data/invasive/test/unknown/1457.jpg
moving from  /home/ubuntu/fastai/data/invasive/test/523.jpg to  /home/ubuntu/fastai/data/invasive

moving from  /home/ubuntu/fastai/data/invasive/test/551.jpg to  /home/ubuntu/fastai/data/invasive/test/unknown/551.jpg
moving from  /home/ubuntu/fastai/data/invasive/test/1439.jpg to  /home/ubuntu/fastai/data/invasive/test/unknown/1439.jpg
moving from  /home/ubuntu/fastai/data/invasive/test/1207.jpg to  /home/ubuntu/fastai/data/invasive/test/unknown/1207.jpg
moving from  /home/ubuntu/fastai/data/invasive/test/19.jpg to  /home/ubuntu/fastai/data/invasive/test/unknown/19.jpg
moving from  /home/ubuntu/fastai/data/invasive/test/1099.jpg to  /home/ubuntu/fastai/data/invasive/test/unknown/1099.jpg
moving from  /home/ubuntu/fastai/data/invasive/test/623.jpg to  /home/ubuntu/fastai/data/invasive/test/unknown/623.jpg
moving from  /home/ubuntu/fastai/data/invasive/test/603.jpg to  /home/ubuntu/fastai/data/invasive/test/unknown/603.jpg
moving from  /home/ubuntu/fastai/data/invasive/test/923.jpg to  /home/ubuntu/fastai/data/invasive/test/unknown/923.jpg
moving from  /home/ubuntu/fastai/data/invasi

moving from  /home/ubuntu/fastai/data/invasive/test/1131.jpg to  /home/ubuntu/fastai/data/invasive/test/unknown/1131.jpg
moving from  /home/ubuntu/fastai/data/invasive/test/1284.jpg to  /home/ubuntu/fastai/data/invasive/test/unknown/1284.jpg
moving from  /home/ubuntu/fastai/data/invasive/test/1372.jpg to  /home/ubuntu/fastai/data/invasive/test/unknown/1372.jpg
moving from  /home/ubuntu/fastai/data/invasive/test/814.jpg to  /home/ubuntu/fastai/data/invasive/test/unknown/814.jpg
moving from  /home/ubuntu/fastai/data/invasive/test/1420.jpg to  /home/ubuntu/fastai/data/invasive/test/unknown/1420.jpg
moving from  /home/ubuntu/fastai/data/invasive/test/531.jpg to  /home/ubuntu/fastai/data/invasive/test/unknown/531.jpg
moving from  /home/ubuntu/fastai/data/invasive/test/1518.jpg to  /home/ubuntu/fastai/data/invasive/test/unknown/1518.jpg
moving from  /home/ubuntu/fastai/data/invasive/test/1438.jpg to  /home/ubuntu/fastai/data/invasive/test/unknown/1438.jpg
moving from  /home/ubuntu/fastai/dat

In [16]:
def splitMoveFiles(fraction, origDir, destDir):
    print categories[0]
    fileList = dirFileList(origDir)
    shuffle(fileList)
    print "Number of files in original directory :", len(fileList)
    num_files_to_move = int(len(fileList) * fraction)
    print "num_files_to_move:", num_files_to_move
    files = fileList[0:num_files_to_move]
    moveFiles(files, origDir, destDir)


In [18]:
#now move validFract from test to valid
#splitMoveFiles(validFract, TRAIN+categories[0], VALID+categories[0])
#done - only do once

harmless/
Number of files in original directory : 847
num_files_to_move: 254
moving from  /home/ubuntu/fastai/data/invasive/train/harmless/2146.jpg to  /home/ubuntu/fastai/data/invasive/valid/harmless/2146.jpg
moving from  /home/ubuntu/fastai/data/invasive/train/harmless/359.jpg to  /home/ubuntu/fastai/data/invasive/valid/harmless/359.jpg
moving from  /home/ubuntu/fastai/data/invasive/train/harmless/1596.jpg to  /home/ubuntu/fastai/data/invasive/valid/harmless/1596.jpg
moving from  /home/ubuntu/fastai/data/invasive/train/harmless/1573.jpg to  /home/ubuntu/fastai/data/invasive/valid/harmless/1573.jpg
moving from  /home/ubuntu/fastai/data/invasive/train/harmless/704.jpg to  /home/ubuntu/fastai/data/invasive/valid/harmless/704.jpg
moving from  /home/ubuntu/fastai/data/invasive/train/harmless/526.jpg to  /home/ubuntu/fastai/data/invasive/valid/harmless/526.jpg
moving from  /home/ubuntu/fastai/data/invasive/train/harmless/1609.jpg to  /home/ubuntu/fastai/data/invasive/valid/harmless/1609.jp

In [20]:
splitMoveFiles(validFract, TRAIN+categories[1], VALID+categories[1])
#done - only do once.

harmless/
Number of files in original directory : 1448
num_files_to_move: 434
moving from  /home/ubuntu/fastai/data/invasive/train/invasive/1449.jpg to  /home/ubuntu/fastai/data/invasive/valid/invasive/1449.jpg
moving from  /home/ubuntu/fastai/data/invasive/train/invasive/617.jpg to  /home/ubuntu/fastai/data/invasive/valid/invasive/617.jpg
moving from  /home/ubuntu/fastai/data/invasive/train/invasive/927.jpg to  /home/ubuntu/fastai/data/invasive/valid/invasive/927.jpg
moving from  /home/ubuntu/fastai/data/invasive/train/invasive/700.jpg to  /home/ubuntu/fastai/data/invasive/valid/invasive/700.jpg
moving from  /home/ubuntu/fastai/data/invasive/train/invasive/1398.jpg to  /home/ubuntu/fastai/data/invasive/valid/invasive/1398.jpg
moving from  /home/ubuntu/fastai/data/invasive/train/invasive/784.jpg to  /home/ubuntu/fastai/data/invasive/valid/invasive/784.jpg
moving from  /home/ubuntu/fastai/data/invasive/train/invasive/1712.jpg to  /home/ubuntu/fastai/data/invasive/valid/invasive/1712.jpg

In [21]:
def copyFiles(fileList, orig, dest):
    success = 0
    error = 0
    count = 0
    for f in fileList:
        filename = orig+f
        destFilename = dest+f
        try:
            print "copying from ", filename+" to ", destFilename
            copyfile(filename, destFilename)
            success += 1
        except:
            print("error copying :", filename)
            error += 1
        count += 1
        #break
    print ("success:", success)
    print ("error:", error)
    print ("count:", count)


In [22]:
def splitCopyFiles(fraction, origDir, destDir):
    print categories[0]
    fileList = dirFileList(origDir)
    shuffle(fileList)
    print "Number of files in original directory :", len(fileList)
    num_files_to_copy = int(len(fileList) * fraction)
    print "num_files_to_copy:", num_files_to_copy
    files = fileList[0:num_files_to_copy]
    copyFiles(files, origDir, destDir)


In [23]:
splitCopyFiles(sampleFract, TRAIN+categories[0], SAMPLE_TRAIN+categories[0])

harmless/
Number of files in original directory : 593
num_files_to_copy: 118
copying from  /home/ubuntu/fastai/data/invasive/train/harmless/588.jpg to  /home/ubuntu/fastai/data/invasive/sample/train/harmless/588.jpg
copying from  /home/ubuntu/fastai/data/invasive/train/harmless/828.jpg to  /home/ubuntu/fastai/data/invasive/sample/train/harmless/828.jpg
copying from  /home/ubuntu/fastai/data/invasive/train/harmless/209.jpg to  /home/ubuntu/fastai/data/invasive/sample/train/harmless/209.jpg
copying from  /home/ubuntu/fastai/data/invasive/train/harmless/1710.jpg to  /home/ubuntu/fastai/data/invasive/sample/train/harmless/1710.jpg
copying from  /home/ubuntu/fastai/data/invasive/train/harmless/1231.jpg to  /home/ubuntu/fastai/data/invasive/sample/train/harmless/1231.jpg
copying from  /home/ubuntu/fastai/data/invasive/train/harmless/182.jpg to  /home/ubuntu/fastai/data/invasive/sample/train/harmless/182.jpg
copying from  /home/ubuntu/fastai/data/invasive/train/harmless/756.jpg to  /home/ubun

copying from  /home/ubuntu/fastai/data/invasive/train/harmless/1789.jpg to  /home/ubuntu/fastai/data/invasive/sample/train/harmless/1789.jpg
copying from  /home/ubuntu/fastai/data/invasive/train/harmless/279.jpg to  /home/ubuntu/fastai/data/invasive/sample/train/harmless/279.jpg
copying from  /home/ubuntu/fastai/data/invasive/train/harmless/2157.jpg to  /home/ubuntu/fastai/data/invasive/sample/train/harmless/2157.jpg
copying from  /home/ubuntu/fastai/data/invasive/train/harmless/1776.jpg to  /home/ubuntu/fastai/data/invasive/sample/train/harmless/1776.jpg
copying from  /home/ubuntu/fastai/data/invasive/train/harmless/2169.jpg to  /home/ubuntu/fastai/data/invasive/sample/train/harmless/2169.jpg
copying from  /home/ubuntu/fastai/data/invasive/train/harmless/331.jpg to  /home/ubuntu/fastai/data/invasive/sample/train/harmless/331.jpg
copying from  /home/ubuntu/fastai/data/invasive/train/harmless/1355.jpg to  /home/ubuntu/fastai/data/invasive/sample/train/harmless/1355.jpg
copying from  /ho

In [24]:
splitCopyFiles(sampleFract, TRAIN+categories[1], SAMPLE_TRAIN+categories[1])

harmless/
Number of files in original directory : 1014
num_files_to_copy: 202
copying from  /home/ubuntu/fastai/data/invasive/train/invasive/151.jpg to  /home/ubuntu/fastai/data/invasive/sample/train/invasive/151.jpg
copying from  /home/ubuntu/fastai/data/invasive/train/invasive/809.jpg to  /home/ubuntu/fastai/data/invasive/sample/train/invasive/809.jpg
copying from  /home/ubuntu/fastai/data/invasive/train/invasive/2120.jpg to  /home/ubuntu/fastai/data/invasive/sample/train/invasive/2120.jpg
copying from  /home/ubuntu/fastai/data/invasive/train/invasive/2018.jpg to  /home/ubuntu/fastai/data/invasive/sample/train/invasive/2018.jpg
copying from  /home/ubuntu/fastai/data/invasive/train/invasive/1531.jpg to  /home/ubuntu/fastai/data/invasive/sample/train/invasive/1531.jpg
copying from  /home/ubuntu/fastai/data/invasive/train/invasive/1058.jpg to  /home/ubuntu/fastai/data/invasive/sample/train/invasive/1058.jpg
copying from  /home/ubuntu/fastai/data/invasive/train/invasive/833.jpg to  /home

copying from  /home/ubuntu/fastai/data/invasive/train/invasive/1002.jpg to  /home/ubuntu/fastai/data/invasive/sample/train/invasive/1002.jpg
copying from  /home/ubuntu/fastai/data/invasive/train/invasive/634.jpg to  /home/ubuntu/fastai/data/invasive/sample/train/invasive/634.jpg
copying from  /home/ubuntu/fastai/data/invasive/train/invasive/626.jpg to  /home/ubuntu/fastai/data/invasive/sample/train/invasive/626.jpg
copying from  /home/ubuntu/fastai/data/invasive/train/invasive/772.jpg to  /home/ubuntu/fastai/data/invasive/sample/train/invasive/772.jpg
copying from  /home/ubuntu/fastai/data/invasive/train/invasive/1517.jpg to  /home/ubuntu/fastai/data/invasive/sample/train/invasive/1517.jpg
copying from  /home/ubuntu/fastai/data/invasive/train/invasive/1220.jpg to  /home/ubuntu/fastai/data/invasive/sample/train/invasive/1220.jpg
copying from  /home/ubuntu/fastai/data/invasive/train/invasive/1305.jpg to  /home/ubuntu/fastai/data/invasive/sample/train/invasive/1305.jpg
copying from  /home

copying from  /home/ubuntu/fastai/data/invasive/train/invasive/1136.jpg to  /home/ubuntu/fastai/data/invasive/sample/train/invasive/1136.jpg
copying from  /home/ubuntu/fastai/data/invasive/train/invasive/1233.jpg to  /home/ubuntu/fastai/data/invasive/sample/train/invasive/1233.jpg
copying from  /home/ubuntu/fastai/data/invasive/train/invasive/2236.jpg to  /home/ubuntu/fastai/data/invasive/sample/train/invasive/2236.jpg
copying from  /home/ubuntu/fastai/data/invasive/train/invasive/1567.jpg to  /home/ubuntu/fastai/data/invasive/sample/train/invasive/1567.jpg
copying from  /home/ubuntu/fastai/data/invasive/train/invasive/478.jpg to  /home/ubuntu/fastai/data/invasive/sample/train/invasive/478.jpg
copying from  /home/ubuntu/fastai/data/invasive/train/invasive/1929.jpg to  /home/ubuntu/fastai/data/invasive/sample/train/invasive/1929.jpg
copying from  /home/ubuntu/fastai/data/invasive/train/invasive/2164.jpg to  /home/ubuntu/fastai/data/invasive/sample/train/invasive/2164.jpg
copying from  /

In [25]:
splitCopyFiles(sampleFract, VALID+categories[0], SAMPLE_VALID+categories[0])

harmless/
Number of files in original directory : 254
num_files_to_copy: 50
copying from  /home/ubuntu/fastai/data/invasive/valid/harmless/2029.jpg to  /home/ubuntu/fastai/data/invasive/sample/valid/harmless/2029.jpg
copying from  /home/ubuntu/fastai/data/invasive/valid/harmless/1609.jpg to  /home/ubuntu/fastai/data/invasive/sample/valid/harmless/1609.jpg
copying from  /home/ubuntu/fastai/data/invasive/valid/harmless/789.jpg to  /home/ubuntu/fastai/data/invasive/sample/valid/harmless/789.jpg
copying from  /home/ubuntu/fastai/data/invasive/valid/harmless/900.jpg to  /home/ubuntu/fastai/data/invasive/sample/valid/harmless/900.jpg
copying from  /home/ubuntu/fastai/data/invasive/valid/harmless/471.jpg to  /home/ubuntu/fastai/data/invasive/sample/valid/harmless/471.jpg
copying from  /home/ubuntu/fastai/data/invasive/valid/harmless/1316.jpg to  /home/ubuntu/fastai/data/invasive/sample/valid/harmless/1316.jpg
copying from  /home/ubuntu/fastai/data/invasive/valid/harmless/1087.jpg to  /home/ub

In [26]:
splitCopyFiles(sampleFract, VALID+categories[1], SAMPLE_VALID+categories[1])

harmless/
Number of files in original directory : 434
num_files_to_copy: 86
copying from  /home/ubuntu/fastai/data/invasive/valid/invasive/993.jpg to  /home/ubuntu/fastai/data/invasive/sample/valid/invasive/993.jpg
copying from  /home/ubuntu/fastai/data/invasive/valid/invasive/1603.jpg to  /home/ubuntu/fastai/data/invasive/sample/valid/invasive/1603.jpg
copying from  /home/ubuntu/fastai/data/invasive/valid/invasive/1645.jpg to  /home/ubuntu/fastai/data/invasive/sample/valid/invasive/1645.jpg
copying from  /home/ubuntu/fastai/data/invasive/valid/invasive/1467.jpg to  /home/ubuntu/fastai/data/invasive/sample/valid/invasive/1467.jpg
copying from  /home/ubuntu/fastai/data/invasive/valid/invasive/1141.jpg to  /home/ubuntu/fastai/data/invasive/sample/valid/invasive/1141.jpg
copying from  /home/ubuntu/fastai/data/invasive/valid/invasive/767.jpg to  /home/ubuntu/fastai/data/invasive/sample/valid/invasive/767.jpg
copying from  /home/ubuntu/fastai/data/invasive/valid/invasive/623.jpg to  /home/u

copying from  /home/ubuntu/fastai/data/invasive/valid/invasive/2145.jpg to  /home/ubuntu/fastai/data/invasive/sample/valid/invasive/2145.jpg
copying from  /home/ubuntu/fastai/data/invasive/valid/invasive/916.jpg to  /home/ubuntu/fastai/data/invasive/sample/valid/invasive/916.jpg
copying from  /home/ubuntu/fastai/data/invasive/valid/invasive/1690.jpg to  /home/ubuntu/fastai/data/invasive/sample/valid/invasive/1690.jpg
copying from  /home/ubuntu/fastai/data/invasive/valid/invasive/2284.jpg to  /home/ubuntu/fastai/data/invasive/sample/valid/invasive/2284.jpg
copying from  /home/ubuntu/fastai/data/invasive/valid/invasive/1916.jpg to  /home/ubuntu/fastai/data/invasive/sample/valid/invasive/1916.jpg
copying from  /home/ubuntu/fastai/data/invasive/valid/invasive/369.jpg to  /home/ubuntu/fastai/data/invasive/sample/valid/invasive/369.jpg
copying from  /home/ubuntu/fastai/data/invasive/valid/invasive/1066.jpg to  /home/ubuntu/fastai/data/invasive/sample/valid/invasive/1066.jpg
copying from  /ho

In [27]:
splitCopyFiles(sampleFract, TEST+UNKNOWN, SAMPLE_TEST+UNKNOWN)

harmless/
Number of files in original directory : 1531
num_files_to_copy: 306
copying from  /home/ubuntu/fastai/data/invasive/test/unknown/1002.jpg to  /home/ubuntu/fastai/data/invasive/sample/test/unknown/1002.jpg
copying from  /home/ubuntu/fastai/data/invasive/test/unknown/1300.jpg to  /home/ubuntu/fastai/data/invasive/sample/test/unknown/1300.jpg
copying from  /home/ubuntu/fastai/data/invasive/test/unknown/408.jpg to  /home/ubuntu/fastai/data/invasive/sample/test/unknown/408.jpg
copying from  /home/ubuntu/fastai/data/invasive/test/unknown/1174.jpg to  /home/ubuntu/fastai/data/invasive/sample/test/unknown/1174.jpg
copying from  /home/ubuntu/fastai/data/invasive/test/unknown/1068.jpg to  /home/ubuntu/fastai/data/invasive/sample/test/unknown/1068.jpg
copying from  /home/ubuntu/fastai/data/invasive/test/unknown/1126.jpg to  /home/ubuntu/fastai/data/invasive/sample/test/unknown/1126.jpg
copying from  /home/ubuntu/fastai/data/invasive/test/unknown/565.jpg to  /home/ubuntu/fastai/data/inva

copying from  /home/ubuntu/fastai/data/invasive/test/unknown/311.jpg to  /home/ubuntu/fastai/data/invasive/sample/test/unknown/311.jpg
copying from  /home/ubuntu/fastai/data/invasive/test/unknown/1360.jpg to  /home/ubuntu/fastai/data/invasive/sample/test/unknown/1360.jpg
copying from  /home/ubuntu/fastai/data/invasive/test/unknown/994.jpg to  /home/ubuntu/fastai/data/invasive/sample/test/unknown/994.jpg
copying from  /home/ubuntu/fastai/data/invasive/test/unknown/1272.jpg to  /home/ubuntu/fastai/data/invasive/sample/test/unknown/1272.jpg
copying from  /home/ubuntu/fastai/data/invasive/test/unknown/1130.jpg to  /home/ubuntu/fastai/data/invasive/sample/test/unknown/1130.jpg
copying from  /home/ubuntu/fastai/data/invasive/test/unknown/219.jpg to  /home/ubuntu/fastai/data/invasive/sample/test/unknown/219.jpg
copying from  /home/ubuntu/fastai/data/invasive/test/unknown/1400.jpg to  /home/ubuntu/fastai/data/invasive/sample/test/unknown/1400.jpg
copying from  /home/ubuntu/fastai/data/invasive

copying from  /home/ubuntu/fastai/data/invasive/test/unknown/650.jpg to  /home/ubuntu/fastai/data/invasive/sample/test/unknown/650.jpg
copying from  /home/ubuntu/fastai/data/invasive/test/unknown/874.jpg to  /home/ubuntu/fastai/data/invasive/sample/test/unknown/874.jpg
copying from  /home/ubuntu/fastai/data/invasive/test/unknown/1449.jpg to  /home/ubuntu/fastai/data/invasive/sample/test/unknown/1449.jpg
copying from  /home/ubuntu/fastai/data/invasive/test/unknown/460.jpg to  /home/ubuntu/fastai/data/invasive/sample/test/unknown/460.jpg
copying from  /home/ubuntu/fastai/data/invasive/test/unknown/893.jpg to  /home/ubuntu/fastai/data/invasive/sample/test/unknown/893.jpg
copying from  /home/ubuntu/fastai/data/invasive/test/unknown/1456.jpg to  /home/ubuntu/fastai/data/invasive/sample/test/unknown/1456.jpg
copying from  /home/ubuntu/fastai/data/invasive/test/unknown/1032.jpg to  /home/ubuntu/fastai/data/invasive/sample/test/unknown/1032.jpg
copying from  /home/ubuntu/fastai/data/invasive/t

copying from  /home/ubuntu/fastai/data/invasive/test/unknown/1009.jpg to  /home/ubuntu/fastai/data/invasive/sample/test/unknown/1009.jpg
copying from  /home/ubuntu/fastai/data/invasive/test/unknown/443.jpg to  /home/ubuntu/fastai/data/invasive/sample/test/unknown/443.jpg
copying from  /home/ubuntu/fastai/data/invasive/test/unknown/641.jpg to  /home/ubuntu/fastai/data/invasive/sample/test/unknown/641.jpg
copying from  /home/ubuntu/fastai/data/invasive/test/unknown/1042.jpg to  /home/ubuntu/fastai/data/invasive/sample/test/unknown/1042.jpg
copying from  /home/ubuntu/fastai/data/invasive/test/unknown/1019.jpg to  /home/ubuntu/fastai/data/invasive/sample/test/unknown/1019.jpg
copying from  /home/ubuntu/fastai/data/invasive/test/unknown/326.jpg to  /home/ubuntu/fastai/data/invasive/sample/test/unknown/326.jpg
copying from  /home/ubuntu/fastai/data/invasive/test/unknown/107.jpg to  /home/ubuntu/fastai/data/invasive/sample/test/unknown/107.jpg
copying from  /home/ubuntu/fastai/data/invasive/t

copying from  /home/ubuntu/fastai/data/invasive/test/unknown/467.jpg to  /home/ubuntu/fastai/data/invasive/sample/test/unknown/467.jpg
copying from  /home/ubuntu/fastai/data/invasive/test/unknown/82.jpg to  /home/ubuntu/fastai/data/invasive/sample/test/unknown/82.jpg
copying from  /home/ubuntu/fastai/data/invasive/test/unknown/1127.jpg to  /home/ubuntu/fastai/data/invasive/sample/test/unknown/1127.jpg
copying from  /home/ubuntu/fastai/data/invasive/test/unknown/10.jpg to  /home/ubuntu/fastai/data/invasive/sample/test/unknown/10.jpg
copying from  /home/ubuntu/fastai/data/invasive/test/unknown/657.jpg to  /home/ubuntu/fastai/data/invasive/sample/test/unknown/657.jpg
copying from  /home/ubuntu/fastai/data/invasive/test/unknown/1028.jpg to  /home/ubuntu/fastai/data/invasive/sample/test/unknown/1028.jpg
copying from  /home/ubuntu/fastai/data/invasive/test/unknown/1088.jpg to  /home/ubuntu/fastai/data/invasive/sample/test/unknown/1088.jpg
copying from  /home/ubuntu/fastai/data/invasive/test/

In [28]:
testFileList = dirFileList(TRAIN+categories[1])
print len(testFileList)

1014


In [6]:
# traverse root directory, and list directories as dirs and files as files
print (DATA_HOME_DIR)
for root, dirs, files in os.walk(DATA_HOME_DIR):
    path = root.split(os.sep)
    print (len(path) - 1) * '-', os.path.basename(root), len(files) 

/home/ubuntu/fastai/data/invasive/
------  6
------ valid 0
------- invasive 434
------- harmless 254
------ results 4
------ train 0
------- invasive 1014
------- harmless 593
------ test 0
------- unknown 1531
------ sample 0
------- valid 0
-------- invasive 86
-------- harmless 50
------- results 1
------- train 0
-------- invasive 202
-------- harmless 118
------- test 0
-------- unknown 306


#use these to revert to original condition  
rm -rf valid  
rm -rf train  
rm -rf test  
rm -rf sample  
